In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
API_key = 'B51wnmtnL30svioyDrNUrhl4h7We2HR8i-DQgKUNPsbVsiruZAItOeM6IHC8e5Y72G-d9TjHZFQHtKJjBrSCWCF0gLTvDlfr9u7bKce5jClTMT4ua-m0dtzxI6q1XXYx'

In [3]:
from yelpapi import YelpAPI
import argparse
from pprint import pprint
#see help code https://github.com/gfairchild/yelpapi/blob/master/examples/examples.py

In [4]:
import requests
import json
headers = {'Authorization': 'Bearer %s' % API_key}

In [5]:
yelp_api = YelpAPI(API_key)

In [6]:
day = int(input("Hello, what's the day of scrapping (day count start at 0) ?")) #day of scrapping

Hello, what's the day of scrapping (day count start at 0) ?2


In [7]:
if day==0: #If this is the first day of scrapping, we open the clean dataset
    data = pd.read_csv('data/chicago-food-inspections/food-inspections_clean.csv')
else: #Else, we open the previously scrapped dataset
    data = pd.read_csv('data/chicago-food-inspections/food-inspections_scrapped'+str(day-1)+'.csv')

In [8]:
data.head(5)

,Unnamed: 0,Unnamed: 0.1,License #,DBA Name,Facility Type,Risk,Address,City,State,Zip,Latitude,Longitude,Inspection Date,Inspection Type,Results,Yelp name,Yelp review,Yelp review count,Yelp rating
0,0,0,2.0,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,CHICAGO,IL,60606.0,41.880757158647214,-87.6347092983425,"['2018-02-13T00:00:00.000', '2017-05-12T00:00:...","['Canvass', 'Canvass', 'Short Form Complaint',...","['Pass w/ Conditions', 'Pass', 'Pass w/ Condit...",Cosi,"[4, 1, 5]",3.0,3.333333
1,1,1,9.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,41.880395838259616,-87.62450172159464,"['2019-08-09T00:00:00.000', '2018-06-19T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass w/ Conditions', 'Pass', 'Fail', 'Pass',...",NaN,NaN,NaN,NaN
2,2,2,40.0,COSI,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,41.886567370886944,-87.62438467059714,"['2018-09-14T00:00:00.000', '2018-03-27T00:00:...","['Complaint', 'Canvass', 'Complaint Re-Inspect...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail',...",Cosi,"[4, 4, 2]",3.0,3.333333
3,3,3,62.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,41.88331785985083,-87.63476909461744,"['2018-02-20T00:00:00.000', '2017-02-09T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass', 'Pass', 'Fail', 'Pass w/ Conditions',...",NaN,NaN,NaN,NaN
4,4,4,85.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),55 E GRAND AVE,CHICAGO,IL,60611.0,41.891590741083505,-87.62586713724458,"['2012-03-28T00:00:00.000', '2011-05-10T00:00:...","['Canvass', 'Canvass', 'Canvass Re-Inspection'...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail']",NaN,NaN,NaN,NaN


Note: We have a bit more than 20'000 rows in our clean dataset, as Yelp API restrict us to 5'000 queries per day, we divided the number of rows by 5, and scrapped the whole dataset in 5 days.

In [9]:
n_days = 5 #Total number of days required
scrapping_per_day = int(data.shape[0]/n_days) #Number of rows to scrapp
if day==5:
    scrapping_per_day = scrapping_per_day + 1 #As the scrapping number is not an interger, we need to add 1 to obtain a complete scrapping

In [11]:
start = scrapping_per_day*day #Starting row for the scrapping
end = start + scrapping_per_day #Ending row for the scrapping
if start == 0: #If this is the first time that we scrapped the data, we need to create the new columns, else, they are already there
    data['Yelp name']=np.NaN
    data['Yelp review']=np.NaN
    data['Yelp review count']=np.NaN
    data['Yelp rating']=np.NaN
data.head()

,Unnamed: 0,Unnamed: 0.1,License #,DBA Name,Facility Type,Risk,Address,City,State,Zip,Latitude,Longitude,Inspection Date,Inspection Type,Results,Yelp name,Yelp review,Yelp review count,Yelp rating
0,0,0,2.0,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,CHICAGO,IL,60606.0,41.880757158647214,-87.6347092983425,"['2018-02-13T00:00:00.000', '2017-05-12T00:00:...","['Canvass', 'Canvass', 'Short Form Complaint',...","['Pass w/ Conditions', 'Pass', 'Pass w/ Condit...",Cosi,"[4, 1, 5]",3.0,3.333333
1,1,1,9.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,41.880395838259616,-87.62450172159464,"['2019-08-09T00:00:00.000', '2018-06-19T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass w/ Conditions', 'Pass', 'Fail', 'Pass',...",NaN,NaN,NaN,NaN
2,2,2,40.0,COSI,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,41.886567370886944,-87.62438467059714,"['2018-09-14T00:00:00.000', '2018-03-27T00:00:...","['Complaint', 'Canvass', 'Complaint Re-Inspect...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail',...",Cosi,"[4, 4, 2]",3.0,3.333333
3,3,3,62.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,41.88331785985083,-87.63476909461744,"['2018-02-20T00:00:00.000', '2017-02-09T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass', 'Pass', 'Fail', 'Pass w/ Conditions',...",NaN,NaN,NaN,NaN
4,4,4,85.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),55 E GRAND AVE,CHICAGO,IL,60611.0,41.891590741083505,-87.62586713724458,"['2012-03-28T00:00:00.000', '2011-05-10T00:00:...","['Canvass', 'Canvass', 'Canvass Re-Inspection'...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail']",NaN,NaN,NaN,NaN


In [12]:
for i in range(start,end):
    temp = data.iloc[i]
    response = yelp_api.business_match_query(name=temp['DBA Name'],
                                             address1=temp['Address'],
                                             city=temp['City'],
                                             state=temp['State'],
                                             country='US') #['businesses'][0]
    
    if len(response['businesses'])>0:
        data['Yelp name'].iloc[i] = response['businesses'][0]['name']
        id_ =  response['businesses'][0]['id']
        url = "https://api.yelp.com/v3/businesses/" + id_ + "/reviews"
        req = requests.get(url, headers=headers)
        #print('the status code is {}'.format(req.status_code))
        parsed = json.loads(req.text)
        rates = [parsed['reviews'][i]['rating'] for i in range(len(parsed['reviews']))]
        data['Yelp rating'].iloc[i] = np.mean(rates)
        data['Yelp review count'].iloc[i] = len(rates)
        data['Yelp review'].iloc[i] = str(rates)
    #print(temp['DBA Name'])
    #pprint(response)

In [13]:
print("Among the rows", start,"and", end,"we scrapped",scrapping_per_day-np.count_nonzero(pd.isnull(data['Yelp name'][start:end])),"restaurants.")

Among the rows 4 and 6 we scrapped 2 restaurants.


In [14]:
#Each day, we save the newly scrapped dataframe to a csv (so if an error occur at one point, we don't loose all the information from the previous days)
data.to_csv("data/chicago-food-inspections/food-inspections_scrapped"+str(day)+".csv")